In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix

In [3]:
data = pd.read_csv('../data/Breast_Cancer.csv')
data

,severity,bdate,marital,breastfa,nobreast,sebreast,nosecon,ovafam,noova,cancerfa,...,childsmo,housesmo,worksmo,alc,nobrefed,dur_brefed,diag_cancer,inj,stopinjm,durinjm
0,Category2,07feb2485,married,No,0.0,No,0.0,No,0.0,No,...,No,Yes,Yes,Never,4,111.0,non-cancer,1.0,NaN,NaN
1,Category2,01nov2486,married,No,0.0,No,0.0,No,0.0,Yes,...,No,No,No,Current user,2,2.0,non-cancer,1.0,NaN,NaN
2,Category2,18may2490,married,No,0.0,No,0.0,No,0.0,No,...,Yes,No,No,Never,2,4.0,non-cancer,1.0,NaN,NaN
3,Category2,08dec2488,window,No,0.0,No,0.0,No,0.0,No,...,No,Yes,No,Never,0,0.0,non-cancer,1.0,NaN,NaN
4,Category2,13dec2489,married,No,0.0,No,0.0,No,0.0,No,...,No,No,No,Never,2,2.0,non-cancer,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15713,Category3,14dec2507,single,No,0.0,No,0.0,No,0.0,No,...,Yes,Yes,No,Never,0,0.0,non-cancer,1.0,NaN,NaN
15714,Category2,01may2495,married,Yes,1.0,No,0.0,No,0.0,No,...,No,No,Yes,Never,2,6.0,non-cancer,1.0,NaN,NaN
15715,Category4a,23nov2506,married,No,0.0,No,0.0,No,0.0,No,...,Yes,Yes,No,Current user,2,12.0,non-cancer,2.0,0.0,6.0
15716,Category2,16nov2518,married,No,0.0,No,0.0,No,0.0,No,...,No,No,No,Current user,1,14.0,non-cancer,1.0,NaN,NaN
